In [10]:
# import pandas as pd

# data = pd.read_json('categorized-comments.jsonl', lines=True)

ValueError: Unmatched ''"' when when decoding 'string'

In [11]:
# import jsonlines
# # blank list
# data = []
 
# with jsonlines.open('categorized-comments.jsonl') as reader:
#     for obj in reader:
#         data.append(obj)

InvalidLineError: line contains invalid json: Unterminated string starting at: line 1 column 31 (char 30) (line 606476)

In [14]:
import jsonlines

data = []
 
with jsonlines.open('categorized-comments.jsonl') as reader:
    for obj in reader.iter(type=dict, skip_invalid=True):
        data.append(obj)

In [15]:
len(data)

606475

In [97]:
data[0:2]

[{'cat': 'sports',
  'txt': "Barely better than Gabbert? He was significantly better this year.  He was 16th in passer rating, ahead of guys like Philip Rivers, Carson Palmer, Eli Manning, and Tyrod Taylor.  And that was with the worst surrounding offensive cast in the league.  He is without a doubt better than the career backups you just listed, and although Jimmy G could potentially be better, he's more risky than Kaep is, especially because we'd only get one cheap year with him anyways, and still have to get rid of much more to acquire him."},
 {'cat': 'sports',
  'txt': 'Fuck the ducks and the Angels! But welcome to all new niners fans!'}]

In [18]:
len(data[0])

2

In [19]:
df = pd.DataFrame(data)

In [20]:
df.head()

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [21]:
df.cat.unique()

array(['sports', 'science_and_technology', 'video_games'], dtype=object)

## Twitter Sentiment Analysis - Learn Python for Data Science #2   
https://www.youtube.com/watch?v=o_OZdbCzHUA

In [23]:
from textblob import TextBlob

In [24]:
wiki = TextBlob(df.txt[0])

In [26]:
wiki.tags[:10]

[('Barely', 'RB'),
 ('better', 'JJR'),
 ('than', 'IN'),
 ('Gabbert', 'NNP'),
 ('He', 'PRP'),
 ('was', 'VBD'),
 ('significantly', 'RB'),
 ('better', 'RBR'),
 ('this', 'DT'),
 ('year', 'NN')]

In [27]:
wiki.words

WordList(['Barely', 'better', 'than', 'Gabbert', 'He', 'was', 'significantly', 'better', 'this', 'year', 'He', 'was', '16th', 'in', 'passer', 'rating', 'ahead', 'of', 'guys', 'like', 'Philip', 'Rivers', 'Carson', 'Palmer', 'Eli', 'Manning', 'and', 'Tyrod', 'Taylor', 'And', 'that', 'was', 'with', 'the', 'worst', 'surrounding', 'offensive', 'cast', 'in', 'the', 'league', 'He', 'is', 'without', 'a', 'doubt', 'better', 'than', 'the', 'career', 'backups', 'you', 'just', 'listed', 'and', 'although', 'Jimmy', 'G', 'could', 'potentially', 'be', 'better', 'he', "'s", 'more', 'risky', 'than', 'Kaep', 'is', 'especially', 'because', 'we', "'d", 'only', 'get', 'one', 'cheap', 'year', 'with', 'him', 'anyways', 'and', 'still', 'have', 'to', 'get', 'rid', 'of', 'much', 'more', 'to', 'acquire', 'him'])

Polarity is how positive or negative something is on a scale of -1 to +1:

In [28]:
wiki.sentiment.polarity

0.24

Subjectivity is how much of an opinion it is vs how factual

In [29]:
wiki.sentiment

Sentiment(polarity=0.24, subjectivity=0.67)

## Sentiment Analysis in 4 Minutes  
https://www.youtube.com/watch?v=AJVP96tAWxw

In [36]:
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier 
# from KaggleWord2VecUtility import KaggleWord2VecUtility
import pandas as pd
import nltk

In [38]:
import re

In [52]:
from nltk.corpus import stopwords

In [48]:
# Remove characters
no_chars = re.sub("[^a-zA-Z]"," ", wiki.string)
no_chars

'Barely better than Gabbert  He was significantly better this year   He was   th in passer rating  ahead of guys like Philip Rivers  Carson Palmer  Eli Manning  and Tyrod Taylor   And that was with the worst surrounding offensive cast in the league   He is without a doubt better than the career backups you just listed  and although Jimmy G could potentially be better  he s more risky than Kaep is  especially because we d only get one cheap year with him anyways  and still have to get rid of much more to acquire him '

In [50]:
# Convert to lowercaase and split
words = no_chars.lower().split()
words[:10]

['barely',
 'better',
 'than',
 'gabbert',
 'he',
 'was',
 'significantly',
 'better',
 'this',
 'year']

In [54]:
# Remove stopwords
stops = set(stopwords.words("english"))
no_stops = [w for w in words if not w in stops]
no_stops[:10]

['barely',
 'better',
 'gabbert',
 'significantly',
 'better',
 'year',
 'th',
 'passer',
 'rating',
 'ahead']

In [55]:
len(words)

93

In [56]:
len(no_stops)

50

Create a bag of words

In [58]:
vectorizer = CountVectorizer(analyzer="word", 
                             tokenizer=None, 
                             preprocessor=None, 
                             stop_words=None, 
                             max_features=5000) #at max 5000 words and their associated frequencies

In [59]:
train_data_features = vectorizer.fit_transform(words)

In [60]:
train_data_features = train_data_features.toarray()

In [61]:
train_data_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [62]:
train_data_features.shape

(93, 65)

Train the classifier (Random Forest here)  
...this may take a while.

In [63]:
forest = RandomForestClassifier(n_estimators=100) # 100 trees

In [66]:
forest = forest.fit( train_data_features, train["sentiment"]) # not sure here

NameError: name 'train' is not defined

In [67]:
clean_test_reviews = []

In [68]:
test_data_features = vectorizer.transform(clean_test_reviews)

In [69]:
test_data_features = test_data_features.toarray()

Predict the category in the testing data

In [ ]:
result = forest.predict(test_data_features)
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv(os.path.join(os.path.dirname(__file__), 'data', 
                           'Bag_of_Words_model.csv'), index=False, quoting=3) # quoting??

## Moving on

### Try some stuff from ATAwP Ch 20 (p 278)

In [72]:
import nltk
import unicodedata

from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin


class TextNormalizer(BaseEstimator, TransformerMixin):

    def __init__(self, language='english'):
        self.stopwords  = set(nltk.corpus.stopwords.words(language))
        self.lemmatizer = WordNetLemmatizer()

    def is_punct(self, token):
        return all(
            unicodedata.category(char).startswith('P') for char in token
        )

    def is_stopword(self, token):
        return token.lower() in self.stopwords

    def normalize(self, document):
        return [
            self.lemmatize(token, tag).lower()
            for paragraph in document
            for sentence in paragraph
            for (token, tag) in sentence
            if not self.is_punct(token) and not self.is_stopword(token)
        ]

    def lemmatize(self, token, pos_tag):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(pos_tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)

    def fit(self, X, y=None):
        return self

    def transform(self, documents):
        for document in documents:
            yield self.normalize(document[0])


def identity(doc):
    return doc

In [73]:
testDF = df.copy()

In [74]:
testDF.head()

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [80]:
import joblib
from sklearn.model_selection import cross_val_score

scoring = https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [95]:
def train_model(model, df=testDF, cv=12):
    # Load corpus data
    X = df.txt
    y = df.cat
    scoring = 'f1' # None #'roc_auc_score' # 'r2_score' # 'f1_score'
    
    # Compute cross validation scores
    scores = cross_val_score(model, X, y, cv=cv, scoring=scoring)
    
    # Fit model on dataset
    model.fit(X, y)
    
    # Return scores
    return scores

In [76]:
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [77]:
regressor = Pipeline([
    ('norm', TextNormalizer()), 
    ('tfidf', TfidfVectorizer()), 
    ('ann', MLPRegressor(hidden_layer_sizes=[500,150], verbose=True))
])

In [96]:
regression_scores = train_model(regressor)

AttributeError: 'TextNormalizer' object has no attribute 'language'

In [85]:
classifier = Pipeline([
    ('norm', TextNormalizer()), 
    ('tfidf', TfidfVectorizer()), 
    ('ann', MLPClassifier(hidden_layer_sizes=[500,150], verbose=True))
])

## Load Data

In [132]:
# Load libraries
import jsonlines
import pandas as pd

In [133]:
%%time
# Load JSON data into a list of dictionaries
data = []
with jsonlines.open('categorized-comments.jsonl') as reader:
    for obj in reader.iter(type=dict, skip_invalid=True):
        data.append(obj)

Wall time: 2.42 s


In [134]:
# Convert data to DataFrame
cat_comments_df = pd.DataFrame(data)
cat_comments_df.head()

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


## Preprocess Text

In [135]:
df = cat_comments_df.copy()

In [102]:
# Load libraries
import re
import sys
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import PorterStemmer

# from sklearn.feature_extraction.text import CountVectorizer

In [103]:
df.head()

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [150]:
punctuation_dict = dict.fromkeys(i for i in range(sys.maxunicode) 
                            if unicodedata.category(chr(i)).startswith('P'))

stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)

def cleanText(string):
    '''Processes string and returns cleaned up list of words'''
    
    # Convert to lowercase
    string = string.lower()
    
    # Remove URLs
    string = re.sub(r'http\S+', '', string)
    
    # Remove punctuation
    string = string.translate(punctuation_dict)
    
    # Remove newlines
    string = string.replace("\n", " ")
    
    # Remove stopwords
    string = [word for word in string.split() if word not in stopwords_dict]
    
    return string

In [151]:
testDF = df.head().copy()
testDF

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [152]:
testDF.txt.apply(lambda x: cleanText(x))

0    [barely, better, gabbert, significantly, bette...
1    [fuck, ducks, angels, welcome, new, niners, fans]
2               [drafted, wrs, matt, millen, probably]
3                                               [done]
4                                                [noo]
Name: txt, dtype: object

In [121]:
string = df.txt[3]
string

'[Done](https://i.imgur.com/2YZ90pm.jpg)'

In [122]:
string = string.lower()
string

'[done](https://i.imgur.com/2yz90pm.jpg)'

In [124]:
string = re.sub(r'http\S+', '', string)
string

'[done]('

In [125]:
string = string.translate(punctuation_dict)
string

'done'

In [126]:
string = string.replace("\n", " ")
string

'done'

In [127]:
string = [word for word in string.split() if word not in stopwords_dict]
string

['done']

In [153]:
testDF = df.head().copy()
testDF

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [154]:
testDF.txt = testDF.txt.apply(lambda x: cleanText(x))
testDF.txt

0    [barely, better, gabbert, significantly, bette...
1    [fuck, ducks, angels, welcome, new, niners, fans]
2               [drafted, wrs, matt, millen, probably]
3                                               [done]
4                                                [noo]
Name: txt, dtype: object

In [156]:
testDF = df.copy()

In [157]:
testDF.txt = testDF.txt.apply(lambda string: cleanText(string))

In [158]:
testDF.head()

,cat,txt
0,sports,"[barely, better, gabbert, significantly, bette..."
1,sports,"[fuck, ducks, angels, welcome, new, niners, fans]"
2,sports,"[drafted, wrs, matt, millen, probably]"
3,sports,[done]
4,sports,[noo]


In [159]:
%%time
# Apply PorterStemmer
porter = PorterStemmer()

testDF.txt = testDF.txt.apply(lambda words: [porter.stem(word) for word in words])

Wall time: 3min 5s


In [160]:
testDF.head()

,cat,txt
0,sports,"[bare, better, gabbert, significantli, better,..."
1,sports,"[fuck, duck, angel, welcom, new, niner, fan]"
2,sports,"[draft, wr, matt, millen, probabl]"
3,sports,[done]
4,sports,[noo]


In [162]:
df.txt = df.txt.apply(lambda x: cleanText(x))

0         [barely, better, gabbert, significantly, bette...
1         [fuck, ducks, angels, welcome, new, niners, fans]
2                    [drafted, wrs, matt, millen, probably]
3                                                    [done]
4                                                     [noo]
                                ...                        
606470    [gtany, chance, install, entire, contents, dis...
606471                                 [probably, happened]
606472    [think, disappointment, came, delay, ps+, vers...
606473    [dishonored, 12, looked, like, arse, hell, sac...
606474                                            [removed]
Name: txt, Length: 606475, dtype: object

In [163]:
df.head()

,cat,txt
0,sports,"[barely, better, gabbert, significantly, bette..."
1,sports,"[fuck, ducks, angels, welcome, new, niners, fans]"
2,sports,"[drafted, wrs, matt, millen, probably]"
3,sports,[done]
4,sports,[noo]


In [164]:
# Load libraries
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk import pos_tag
from sklearn.preprocessing import MultiLabelBinarizer


#### CountVectorizer() documentation:
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [165]:
count = CountVectorizer()

In [180]:
df.head().txt

0    [barely, better, gabbert, significantly, bette...
1    [fuck, ducks, angels, welcome, new, niners, fans]
2               [drafted, wrs, matt, millen, probably]
3                                               [done]
4                                                [noo]
Name: txt, dtype: object

In [182]:
sparseWCV = count.fit_transform(df.txt[0])
sparseWCV

<52x46 sparse matrix of type '<class 'numpy.int64'>'
	with 51 stored elements in Compressed Sparse Row format>

Note this step only works with one list, not an array of lists.  
So, they need to be combined into one big list:

In [193]:
text_data, string = [], " "

for text in df.head(100).txt:
    text_data.append(string.join(text))

In [195]:
text_data[:5]

['barely better gabbert significantly better year 16th passer rating ahead guys like philip rivers carson palmer eli manning tyrod taylor worst surrounding offensive cast league without doubt better career backups listed although jimmy g could potentially better hes risky kaep especially wed get one cheap year anyways still get rid much acquire',
 'fuck ducks angels welcome new niners fans',
 'drafted wrs matt millen probably',
 'done',
 'noo']

In [196]:
sparseWCV = count.fit_transform(text_data)
sparseWCV

<100x826 sparse matrix of type '<class 'numpy.int64'>'
	with 1333 stored elements in Compressed Sparse Row format>

In [197]:
sparseWCV.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [198]:
sparseWCV.toarray().shape

(100, 826)

In [199]:
len(sparseWCV.toarray()[1])

826

In [200]:
len(count.get_feature_names())

826

In [ ]:
from multiprocessing import Pool

pool = Pool(processes=4)

classifier = MLPClassifier()
output = pool.map(classifier.fit, (X_train, y_train))